# Dev.scrap - Revisão

---

Este notebook visa fazer uma revisão dos conteúdos dos Capítulos 1 até 4 do livro "Web Scraping com Python" da Ryan Mitchell.

*Observação*: É uma boa atualizar esse notebook para cada começo de ano para continuar a frente sem ter que voltar do zero.

In [7]:
x = 2

In [8]:
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import random

## Capítulo 1 e 2

No capítulo 1, aprendemos o básico sobre como obter uma página web a partir de um código Python e transformá-lo em um objeto BeautifulSoup para podermos navegar facilmente pelas tags, classes e tudo o que compõe o HTML.

Já no capítulo 2, aprendemos a usar o Beautiful Soup e todas as ferramentas que ele nos tem a oferecer, além disso, aprendemos um pouco sobre expressões regulares.

Caso queira treinar os conhecimentos deste capítulo, utilize este [notebook](https://github.com/USPCodeLabSanca/dev-scrap-book-solutions/blob/main/caps%201-2/chapter1-2.ipynb).

Para obter uma página web e transformá-la em um objeto BeautifulSoup, faça o seguinte:

In [9]:
def getPage(url):
  try:
    html = urlopen(url)
  except HTTPError:
    return None
  except URLError:
    return None
  return html

def getBS(url):
  page = getPage(url)
  if page:
    return BeautifulSoup(getPage(url).read(), 'html.parser')
  return None

In [10]:
site = getBS("https://scraping-cap1-2.netlify.app/")
site

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>Caps 1-2</title>
</head>
<style>

    body {
        margin: 0;
        padding: 1em 1em 2em 1em;
    }
    table {
        margin: 1em 0em;
        font-family: arial, sans-serif;
        border-collapse: collapse;
        width: 100%;
    }
    
    td, th {
        border: 1px solid #dddddd;
        text-align: left;
        padding: 8px;
    }
    
    .oscar {
        background-color: gold !important; 
    }
    
    tr:nth-child(even) {
      background-color: #dddddd;
    }

    img {
        width: 15%;
        max-height: 5%;
    }

</style>
<body>
<h1>Movie List by Production Companies</h1>
<div id="warner">
<h2>Warner Bros. Pictures</h2>
<img alt="warner" src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Warner_Bros._%282019%29_logo.svg/1200px-Warner_Bros._%282019%2

Para acessar o primeiro de um tipo de tag no escopo em que está, podemos fazer de duas maneiras:

In [11]:
print(site.h2)
print(site.find('h2'))

<h2>Warner Bros. Pictures</h2>
<h2>Warner Bros. Pictures</h2>


Se quiser acessar o valor textual da tag, use `.get_text()`

In [12]:
print(site.h2.get_text())

Warner Bros. Pictures


Para encontrar determinadas tags a partir de uma pesquisa, utilizaremos principalmente dois métodos com definições parecidas:
* `find_all(tag, attributes, recursive, text, limit, keywords)`
* `find(tag, attributres, recursive, text, keywords)`

Com o parâmetro `tag`, podemos passar uma tag ou uma lista de tags:

In [13]:
print(site.find('h2'))
print(site.find_all('h2'))

print(site.find(['h1', 'h2'])) # Retorna a primeira tag encontrada dentre as passadas na lista
print(site.find_all(['h1', 'h2']))

<h2>Warner Bros. Pictures</h2>
[<h2>Warner Bros. Pictures</h2>, <h2>Participant Films</h2>]
<h1>Movie List by Production Companies</h1>
[<h1>Movie List by Production Companies</h1>, <h2>Warner Bros. Pictures</h2>, <h2>Participant Films</h2>]


Com o parâmetro `attributes`, é passado um dicionário que faz a correspondência com tags que contenham qualquer um desses atributos.

In [14]:
print(site.find_all('tr', {'class': {'oscar', 'cannes'}})) # Retornam todas as tags tr que tenham classe 'oscar' OU 'cannes' 

[<tr class="oscar">
<td>Green Book</td>
<td>Viggo Mortensen, Mahershala Ali, Linda Cardellini</td>
<td>Peter Farrelly</td>
<td>2018</td>
</tr>, <tr class="oscar">
<td>The Post</td>
<td>Meryl Streep, Tom Hanks, Sarah Paulson</td>
<td>Steven Spielberg</td>
<td>2017</td>
</tr>, <tr class="oscar">
<td>Roma</td>
<td>Yalitza Aparicio, Marina de Tavira</td>
<td>Alfonso Cuarón</td>
<td>2018</td>
</tr>, <tr class="oscar">
<td>Spotlight</td>
<td>Mark Rufallo, Michael Keaton, Rachel McAdams</td>
<td>Tom McCarthy</td>
<td>2015</td>
</tr>]


O parâmetro `recursive` (`True` por default) detalha o nível de aprofundamento que o `find` ou `find_all` pode ter. Caso esteja setado para `True`, pode vasculhar toda a árvore do DOM para baixo. Caso contrário, analisa apenas o nível mais superficial.

In [15]:
print(site.find('td', recursive=False))
print(site.find('td', recursive=True))
print(site.find('td'))

None
<td>Harry Potter and the Philosopher's Stone</td>
<td>Harry Potter and the Philosopher's Stone</td>


O parâmetro `string` (`text` está depreciado) faz uma busca no conteúdo textual das tags.

In [16]:
print(site.find(string="Green Book"))

Green Book


O parâmetro `limit` é usado somente no `find_all` e retorna os $n$ primeiros resultados especificados pela pesquisa.

In [17]:
print(site.find_all('td', limit=2))
print(len(site.find_all('td')))

[<td>Harry Potter and the Philosopher's Stone</td>, <td>Daniel Radcliffe, Emma Watson, Rupert Grint, Alan Rickman, Richard Harris</td>]
52


O parâmetro `keyword` permite definir que os resultados deverão ter atributos com os seguintes valores.

Diferentemente do `attributes` que funciona com um OR, o keyword funciona com um AND.

In [18]:
print(site.find(id="warner"))
print(site.find(class_="oscar"))

<div id="warner">
<h2>Warner Bros. Pictures</h2>
<img alt="warner" src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Warner_Bros._%282019%29_logo.svg/1200px-Warner_Bros._%282019%29_logo.svg.png"/>
<table>
<tr>
<th>Movie</th>
<th>Actors</th>
<th>Year</th>
<th>Director</th>
</tr>
<tr>
<td>Harry Potter and the Philosopher's Stone</td>
<td>Daniel Radcliffe, Emma Watson, Rupert Grint, Alan Rickman, Richard Harris</td>
<td>2001</td>
<td>Chris Columbus</td>
</tr>
<tr>
<td>Harry Potter and the Chamber of Secrets</td>
<td>Daniel Radcliffe, Emma Watson, Rupert Grint, Alan Rickman, Richard Harris</td>
<td>2002</td>
<td>Chris Columbus</td>
</tr>
<tr>
<td>Harry Potter and the Prisoner of Askaban</td>
<td>Daniel Radcliffe, Emma Watson, Rupert Grint, Alan Rickman, Michael Gambon</td>
<td>2004</td>
<td>Alfonso Cuarón</td>
</tr>
<tr>
<td>Harry Potter and the Goblet of Fire</td>
<td>Daniel Radcliffe, Emma Watson, Rupert Grint, Alan Rickman, Michael Gambon</td>
<td>2005</td>
<td>Mike Newell<

Para conseguir todas as tags filhas de uma tag, você pode usar o `.children`. 

Ele gera um iterador, caso você queria uma lista, pode usar o `.find_all(recursive=False)`.

*Observação*: note que o `child` no código abaixo retorna umas linhas vazias, então fique atento a isso ao usar o `.children`

In [19]:
participant = site.find(id='participant')
for i, child in enumerate(participant.children):
  print(i, child)

print(participant.find_all(recursive=False))

0 

1 <h2>Participant Films</h2>
2 

3 <img alt="participant" src="https://upload.wikimedia.org/wikipedia/commons/0/07/Participant_%282019%29.svg"/>
4 

5 <table>
<tr>
<th>Movie</th>
<th>Actors</th>
<th>Director</th>
<th>Year</th>
</tr>
<tr class="oscar">
<td>Green Book</td>
<td>Viggo Mortensen, Mahershala Ali, Linda Cardellini</td>
<td>Peter Farrelly</td>
<td>2018</td>
</tr>
<tr class="oscar">
<td>The Post</td>
<td>Meryl Streep, Tom Hanks, Sarah Paulson</td>
<td>Steven Spielberg</td>
<td>2017</td>
</tr>
<tr class="oscar">
<td>Roma</td>
<td>Yalitza Aparicio, Marina de Tavira</td>
<td>Alfonso Cuarón</td>
<td>2018</td>
</tr>
<tr class="oscar">
<td>Spotlight</td>
<td>Mark Rufallo, Michael Keaton, Rachel McAdams</td>
<td>Tom McCarthy</td>
<td>2015</td>
</tr>
</table>
6 

[<h2>Participant Films</h2>, <img alt="participant" src="https://upload.wikimedia.org/wikipedia/commons/0/07/Participant_%282019%29.svg"/>, <table>
<tr>
<th>Movie</th>
<th>Actors</th>
<th>Director</th>
<th>Year</th>
</tr>


Para conseguir todas as tags descendentes de uma tag, você pode usar o `.descendants`. 

Ele gera um iterador, caso você queria uma lista, pode usar o `.find_all()`.

In [20]:
# for i, child in enumerate(participant.descendants):
#   print(i, child)

print(participant.find_all())

[<h2>Participant Films</h2>, <img alt="participant" src="https://upload.wikimedia.org/wikipedia/commons/0/07/Participant_%282019%29.svg"/>, <table>
<tr>
<th>Movie</th>
<th>Actors</th>
<th>Director</th>
<th>Year</th>
</tr>
<tr class="oscar">
<td>Green Book</td>
<td>Viggo Mortensen, Mahershala Ali, Linda Cardellini</td>
<td>Peter Farrelly</td>
<td>2018</td>
</tr>
<tr class="oscar">
<td>The Post</td>
<td>Meryl Streep, Tom Hanks, Sarah Paulson</td>
<td>Steven Spielberg</td>
<td>2017</td>
</tr>
<tr class="oscar">
<td>Roma</td>
<td>Yalitza Aparicio, Marina de Tavira</td>
<td>Alfonso Cuarón</td>
<td>2018</td>
</tr>
<tr class="oscar">
<td>Spotlight</td>
<td>Mark Rufallo, Michael Keaton, Rachel McAdams</td>
<td>Tom McCarthy</td>
<td>2015</td>
</tr>
</table>, <tr>
<th>Movie</th>
<th>Actors</th>
<th>Director</th>
<th>Year</th>
</tr>, <th>Movie</th>, <th>Actors</th>, <th>Director</th>, <th>Year</th>, <tr class="oscar">
<td>Green Book</td>
<td>Viggo Mortensen, Mahershala Ali, Linda Cardellini</td>


Para chamar os irmãos seguintes a uma tag, usamos `.next_siblings`:

In [21]:
the_post = participant.find_all('tr')[2]
for sibling in the_post.next_siblings:
  print(sibling)



<tr class="oscar">
<td>Roma</td>
<td>Yalitza Aparicio, Marina de Tavira</td>
<td>Alfonso Cuarón</td>
<td>2018</td>
</tr>


<tr class="oscar">
<td>Spotlight</td>
<td>Mark Rufallo, Michael Keaton, Rachel McAdams</td>
<td>Tom McCarthy</td>
<td>2015</td>
</tr>




Para chamar os irmãos anteriores a uma tag, usamos `.previous_siblings`:

In [22]:
for sibling in the_post.previous_siblings:
  print(sibling)



<tr class="oscar">
<td>Green Book</td>
<td>Viggo Mortensen, Mahershala Ali, Linda Cardellini</td>
<td>Peter Farrelly</td>
<td>2018</td>
</tr>


<tr>
<th>Movie</th>
<th>Actors</th>
<th>Director</th>
<th>Year</th>
</tr>




Para obter a tag pai de uma determinada tag, utilizamos o `.parent`:

In [23]:
the_post.parent

<table>
<tr>
<th>Movie</th>
<th>Actors</th>
<th>Director</th>
<th>Year</th>
</tr>
<tr class="oscar">
<td>Green Book</td>
<td>Viggo Mortensen, Mahershala Ali, Linda Cardellini</td>
<td>Peter Farrelly</td>
<td>2018</td>
</tr>
<tr class="oscar">
<td>The Post</td>
<td>Meryl Streep, Tom Hanks, Sarah Paulson</td>
<td>Steven Spielberg</td>
<td>2017</td>
</tr>
<tr class="oscar">
<td>Roma</td>
<td>Yalitza Aparicio, Marina de Tavira</td>
<td>Alfonso Cuarón</td>
<td>2018</td>
</tr>
<tr class="oscar">
<td>Spotlight</td>
<td>Mark Rufallo, Michael Keaton, Rachel McAdams</td>
<td>Tom McCarthy</td>
<td>2015</td>
</tr>
</table>

Para relembrar a sintaxe do regex (expressão regular), utilize a página 45 do livro.
Para usar o regex, utilize o `re.compile()`:

In [24]:
# site.find_all('img', {'src': re.compile('.*svg$')})
site.find_all('td', text=re.compile('The Post'))[0].parent

<tr class="oscar">
<td>The Post</td>
<td>Meryl Streep, Tom Hanks, Sarah Paulson</td>
<td>Steven Spielberg</td>
<td>2017</td>
</tr>

Para acessar atributos de uma tag usamos `.attrs`

In [25]:
site.html.attrs

{'lang': 'en'}

Para utilizar expressões lambda, precisamos que elas esperem um parâmetro do tipo BeautifulSoup e que retornem um valor booleano. Dessa forma, o `find` e o `find_all` filtram a partir do resultado da expressão.

In [26]:
site.find_all(lambda tag: len(tag.attrs) <= 2)

[<html lang="en">
 <head>
 <meta charset="utf-8"/>
 <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
 <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
 <title>Caps 1-2</title>
 </head>
 <style>
 
     body {
         margin: 0;
         padding: 1em 1em 2em 1em;
     }
     table {
         margin: 1em 0em;
         font-family: arial, sans-serif;
         border-collapse: collapse;
         width: 100%;
     }
     
     td, th {
         border: 1px solid #dddddd;
         text-align: left;
         padding: 8px;
     }
     
     .oscar {
         background-color: gold !important; 
     }
     
     tr:nth-child(even) {
       background-color: #dddddd;
     }
 
     img {
         width: 15%;
         max-height: 5%;
     }
 
 </style>
 <body>
 <h1>Movie List by Production Companies</h1>
 <div id="warner">
 <h2>Warner Bros. Pictures</h2>
 <img alt="warner" src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Warner_Bros._%282019%29_logo.svg/1

In [27]:
exp_lambda = lambda tag: len(tag.attrs) >= 2

exp_lambda(site.html)

False

## Capítulo 3

No capítulo 3, aprendemos a escrever nossos primeiros web crawlers. Ou seja, sabemos agora como pular de um site para outro, rastrear um domínio e criar um caminhante aleatório por um domínio ou por um site.

Caso queira treinar os conhecimentos deste capítulo, utilize este [notebook](https://github.com/USPCodeLabSanca/dev-scrap-book-solutions/blob/main/cap%203/chapter-3.ipynb).

Vamos utilizar o site https://scraping-cap3.netlify.app/ nos próximos blocos.

Façamos uma função que obtenha todos os links internos presentes em uma página desse website.

Obs: perceba que utilizamos um set, pois não queremos links repetidos na nossa lista. No entanto, um link do tipo "https://scraping-cap3.netlify.app/baixistas.html" é igual à "./baixistas.html" na prática, porém nosso set não irá diferenciar entre os dois.

In [32]:
[2*i for i in range(3)]

[0, 2, 4]

In [34]:
def getInternalLinksOfPage(bsObject):
  if bsObject is None:
    return set()
  links = bsObject.find_all('a', {'href': re.compile('(^\.*\/.+$)|(https:\/\/scraping-cap3\.netlify\.app\/.*)')})
  links = set([link.attrs['href'] for link in links])
  return links

site = getBS("https://scraping-cap3.netlify.app/")
getInternalLinksOfPage(site)

{'/baixistas',
 '/bandas',
 '/bateristas',
 '/guitarristas',
 '/tecladistas',
 '/vocalistas'}

Vamos fazer uma função que torna nossos links internos em formato de URL. Para que possamos ter um formato padrão para o nosso set.

In [29]:
def makeInternalLinkAnURL(link):
  url = 'https://scraping-cap3.netlify.app/'
  if link.startswith('./'):
    url += link[2:]
  elif link.startswith('/'):
    url += link[1:]
  elif link.startswith('../'):  # isso está incorreto, mas vamos deixar assim por facilidade
    url += link[3:]
  else:
    return link
    
  return url

makeInternalLinkAnURL("./guitarristas.html")

'https://scraping-cap3.netlify.app/guitarristas.html'

Um domínio funciona como um grafo direcionado, onde as setas representam os direcionamentos de uma página para a outra. Na verdade, toda a internet funciona dessa forma. 

Para navegarmos a totalidade de um grafo, um algoritmo usual para fazermos isso é uma BFS (*Breadth-first Search*).

<img src="https://www.guru99.com/images/1/020820_0543_BreadthFirs1.png" alt="BFS diagram"/>

O algoritmo funciona da seguinte maneira:
1. Temos uma fila que indica a ordem em que iremos visitar cada nó

2. Temos um hash-set com os nós já visitados

3. Iniciamos a fila com um nó inicial e o hash-set vazio

4. A cada iteração
  1. Retiramos e obtemos o nó da fila
  2. Realizamos alguma operação sobre o nó
  3. Adicionamos o nó ao hash-set
  4. Adicionamos os nós adjacentes que não estão no hash-set na fila

5. Itere enquanto a fila não estiver vazia

Vamos fazer uma BFS para navegar pelo domínio:

In [35]:
def bfs(inicial):
  fila = [makeInternalLinkAnURL(inicial)]
  visitados = set()

  while len(fila) > 0:
    no = fila.pop()
    visitados.add(no)
    for link in getInternalLinksOfPage(getBS(no)):
      url = makeInternalLinkAnURL(link)
      if url not in visitados:
        fila.append(url)

  return visitados

bfs("https://scraping-cap3.netlify.app/")

{'https://scraping-cap3.netlify.app/',
 'https://scraping-cap3.netlify.app/baixistas',
 'https://scraping-cap3.netlify.app/baixistas/dee_dee_ramone',
 'https://scraping-cap3.netlify.app/baixistas/geezer_butler',
 'https://scraping-cap3.netlify.app/baixistas/john_paul_jones',
 'https://scraping-cap3.netlify.app/baixistas/krist_novoselic',
 'https://scraping-cap3.netlify.app/baixistas/paul_mccartney',
 'https://scraping-cap3.netlify.app/baixistas/roger_glover',
 'https://scraping-cap3.netlify.app/baixistas/simon_gallup',
 'https://scraping-cap3.netlify.app/bandas',
 'https://scraping-cap3.netlify.app/bandas/black_sabbath',
 'https://scraping-cap3.netlify.app/bandas/deep_purple',
 'https://scraping-cap3.netlify.app/bandas/led_zeppelin',
 'https://scraping-cap3.netlify.app/bandas/nirvana',
 'https://scraping-cap3.netlify.app/bandas/ramones',
 'https://scraping-cap3.netlify.app/bandas/the_beatles',
 'https://scraping-cap3.netlify.app/bandas/the_cure',
 'https://scraping-cap3.netlify.app/ban

Para pegarmos os links externos de um site, pegamos todos os links que não são internos. Para isso, utilizamos a operação de diferença da teoria de conjuntos.

In [36]:
def getExternalLinksOfPage(bsObject):
  if bsObject is None:
    return set()
  aTags = bsObject.find_all('a')
  links = set([tag.attrs['href'] for tag in aTags if 'href' in tag.attrs])
  links.difference_update(getInternalLinksOfPage(bsObject))
  links = {link for link in links if len(link) > 0}
  links = {link for link in links if link[0] == "h"}
  return links

In [37]:
getExternalLinksOfPage(getBS("https://scraping-cap3.netlify.app/"))

{'https://github.com/kibonusp',
 'https://www.linkedin.com/in/gabrielfreitas-xv/'}

Podemos fazer um caminhante aleatório pela internet a partir disso. Você poderia impedir ele de visitar páginas novamente utilizando um set, porém teria que transformar links internos em URLs, mas tô com preguiça pra isso 😜

In [39]:
def makeInternalLinkAnURL(link, hostname):
  url = hostname
  if link.startswith('./'):
    url += link[2:]
  elif link.startswith('/'):
    url += link[1:]
  elif link.startswith('../'):  # isso está incorreto, mas vamos deixar assim por facilidade
    url += link[3:]
  else:
    return link
    
  return url

In [40]:
o =urlparse("https://scraping-cap3.netlify.app/conversa")
o.hostname

'scraping-cap3.netlify.app'

In [41]:
def bfsExternalLink(page, hostname):
  url = makeInternalLinkAnURL(page, hostname)
  external_links = getExternalLinksOfPage(getBS(url))
  if len(external_links) > 0:
      return url, external_links

  fila = [url]
  visitados = set()

  while len(fila) > 0:
    no = fila.pop()
    visitados.add(no)
    external_links = getExternalLinksOfPage(getBS(no))
    if external_links:
      return no, external_links
    for link in getInternalLinksOfPage(getBS(no)):
      url = makeInternalLinkAnURL(link)
      if url not in visitados:
        fila.append(url)

  return None, set()


In [42]:
def random_walker(link):
  print(link)
  visited = {link}
  linkp = urlparse(link)
  url, external_links = bfsExternalLink(link, linkp.hostname)
  if len(external_links) == 0:
    print("No external links found")
    return {link}
  next_link = random.choice(list(external_links))
  visited.add(next_link)
  return visited.union(random_walker(next_link))

In [43]:
random_walker("https://scraping-cap3.netlify.app")

https://scraping-cap3.netlify.app
https://github.com/kibonusp
https://github.com/kibonusp?tab=followers
https://docs.github.com/en/articles/blocking-a-user-from-your-personal-account
https://github.com/pricing
https://github.com/github-copilot/business_signup
https://github.com/contact
https://twitter.com/github
No external links found


{'https://docs.github.com/en/articles/blocking-a-user-from-your-personal-account',
 'https://github.com/contact',
 'https://github.com/github-copilot/business_signup',
 'https://github.com/kibonusp',
 'https://github.com/kibonusp?tab=followers',
 'https://github.com/pricing',
 'https://scraping-cap3.netlify.app',
 'https://twitter.com/github'}